In [ ]:
import torch
import torchvision
import random
import numpy as np


class MNIST01:
    def __init__(self):

        self.train_samples_class0 = []
        self.train_samples_class1 = []

        batch_size = 32 # samples per batch
        num_classes = 2

        # 32 samples per batch / 2 number of classes = ((32 / 2) * (samples / batch)) / class
        # = (16 samples per batch) / class
        # = 16 samples_per_batch_per_class

        self.samples_per_batch_per_class = int(batch_size / num_classes)

        input_nodes = 784
        l1_nodes = 1024
        l2_nodes = 32
        l3_nodes = 1

        # create batch of samples, batch_size is number of images per batch
        # or number of rows in the batch matrix.
        # each element in a row is a gray scale pixel value from 0 to 255
        self.x_batch = torch.zeros((batch_size, input_nodes))
        
        self.w1 = torch.randn(input_nodes, l1_nodes)
        self.w1 /= input_nodes**0.5

        self.b1 = torch.zeros(l1_nodes)
        
        self.w2 = torch.randn(l1_nodes, l2_nodes)
        self.w2 /= l1_nodes**0.5

        self.w3 = torch.randn(l2_nodes, l3_nodes)
        self.w3 /= l2_nodes**0.5

        self.lr = 0.001
        
    def img_to_tensor_batch(self):
        # 60,000 samples, 28x28 pixels / sample
        self.train_dataset = torchvision.datasets.MNIST(train=True, download=True, root='./')
        
        # 10,000 samples, 28x28 pixels / sample
        self.test_dataset = torchvision.datasets.MNIST(train=False, download=True, root='./')
        
        # --------- Training dataset preparation ---------
        # Filter: class0 = 0, class1 = 1
        for i in range(len(self.train_dataset)):
            # if the img label is a 0, put the image tensor in the list of class0 samples
            # else if it's a 1, put it in the list of class1 samples
            if self.train_dataset[i][1] == 0:
                self.train_samples_class0.append(torch.tensor(np.array(self.train_dataset[i][0]).flatten()))
            elif self.train_dataset[i][1] == 1:
                self.train_samples_class1.append(torch.tensor(np.array(self.train_dataset[i][0]).flatten()))

        # stack all of the rows of tensors from the list, into a single tensor
        self.train_samples_class0_stacked = torch.stack(self.train_samples_class0)
        self.train_samples_class1_stacked = torch.stack(self.train_samples_class1)
        # --------------------------------------------------

        # --------- Test dataset preparation ---------------
        self.test_samples_class0 = []
        self.test_samples_class1 = []

        # Filter: class0 = 0, class1 = 1
        for i in range(len(self.test_dataset)):
            # if the img label is a 0, put the image tensor in the list of class0 samples
            # else if it's a 1, put it in the list of class1 samples
            if self.test_dataset[i][1] == 0:
                self.test_samples_class0.append(torch.tensor(np.array(self.test_dataset[i][0]).flatten()))
            elif self.test_dataset[i][1] == 1:
                self.test_samples_class1.append(torch.tensor(np.array(self.test_dataset[i][0]).flatten()))
        
        # stack all of the rows of tensors from the list, into a single tensor
        self.test_samples_class0_stacked = torch.stack(self.test_samples_class0)
        self.test_samples_class1_stacked = torch.stack(self.test_samples_class1)

    def train(self):
        # ------ forward -------------
        for _epoch in range(1):
            # grab a batch from, and process it
            for _batch in range(int(20000)):
                class0 = self.train_samples_class0_stacked[torch.randperm(len(self.train_samples_class0_stacked))[:int(self.samples_per_batch_per_class)]] / 255
                class1 = self.train_samples_class1_stacked[torch.randperm(len(self.train_samples_class1_stacked))[:int(self.samples_per_batch_per_class)]] / 255

                self.x_batch = torch.cat([class0, class1])
                y = torch.cat([torch.zeros(self.samples_per_batch_per_class, dtype=torch.float),
                               torch.ones(self.samples_per_batch_per_class, dtype=torch.float)])
                
                y = y.view(-1,1)
                zero = torch.tensor(0.0)

                h1 = self.x_batch@self.w1 + self.b1
                relu1 = torch.maximum(zero,h1)
                
                h2 = relu1@self.w2
                relu2 = torch.maximum(zero,h2)

                h3 = relu2@self.w3
                sig3 = 1 / (1 + torch.e**(-h3))

                mse_loss = ((sig3 - y)**2).mean()

                # ------ backward ---------------
                dsig3 = 2 * (sig3 - y)
                dh3 = sig3 * (1 - sig3) * dsig3
                dh3 = dh3
                dsig2 = dh3@self.w3.T
                dsig2 = dsig2
                dw3 = relu2.T@dh3
                dh2 = (h2 > 0).float() * dsig2
                dsig1 = dh2@self.w2.T 
                dw2 = relu1.T@dh2
                dh1 = (h1 > 0).float() * dsig1
                dw1 = self.x_batch.T@dh1
                db1 = dh1.sum(dim=0)
                
                # ------ update ---------------
                self.w1 = self.w1 - self.lr * dw1
                self.w2 = self.w2 - self.lr * dw2
                self.w3 = self.w3 - self.lr * dw3
                self.b1 = self.b1 - self.lr * db1

                print(f'{mse_loss}')

    def inference(self,x):
        x = x / 255
        x = x.view(1,-1)

        h1 = x@self.w1 + self.b1

        zero = torch.tensor(0.0)
        relu1 = torch.maximum(zero,h1)
        
        h2 = relu1@self.w2
        relu2 = torch.maximum(zero,h2)

        h3 = relu2@self.w3
        sig3 = 1 / (1 + torch.e**(-h3))

        return round(sig3.item(),1)
    
    def test(self):
        class0_count = 0
        class1_count = 0

        iterations = 10000
        for i in range(iterations):
            # inputs random class0 img to model, gets prediction
            r = random.randint(0,800)
            prediction = self.inference(self.test_samples_class0_stacked[r])  # class0 = 0, class1 = 1

            # if prediction is class0, add to count
            if prediction > 0.0 and prediction <= 0.5:
                class0_count += 1
            elif prediction > 0.5 and prediction <= 1.0:
                class1_count += 1


        print(f'accuracy %: {class0_count, class1_count}')


a = MNIST01()
a.img_to_tensor_batch()
# a.train()

In [ ]:
a.test() # 10,000 tests

In [ ]:
# Send inference result to device
# import socket
# s = socket.socket()
# s.connect(('192.168.4.1', 80))
# s.send(b'0.5')
# s.close()

In [ ]:
import cv2
import torch
from time import sleep
from PIL import Image
# import socket
# sleep(4)

cam = cv2.VideoCapture(0)
ret, frame = cam.read()
tensor = torch.from_numpy(frame)
cam.release()

img = Image.fromarray(frame)
im = img.resize((28,28))
im = torch.tensor(np.array(im))
out = a.inference(im)

# send result to esp32
# s = socket.socket()
# s.connect(('192.168.4.1', 80))
# print(out)
# s.send(str(out).encode())
# s.close()


prediction = 0 if out < 0.5 else 1
print(f'Prediction: {prediction}\n {out=}')
img

In [ ]:
# rewriting from scratch again parts
import torch
import torchvision
import random
import numpy as np
import cv2


class MNIST01Part2:
    def __init__(self):
        pass

    def get_datasets(self):
        # 60,000 samples, 28x28 pixels / sample
        self.train_dataset = torchvision.datasets.MNIST(train=True, download=True, root='./')
        
        # 10,000 samples, 28x28 pixels / sample
        self.test_dataset = torchvision.datasets.MNIST(train=False, download=True, root='./')
        
    def convert_datasets_to_tensors(self):
        self.train_imgs = self.train_dataset.data.reshape(-1,784)
        self.train_labels = self.train_dataset.targets

        self.test_imgs = self.test_dataset.data.reshape(-1,784)
        self.test_labels = self.test_dataset.targets

In [ ]:
b = MNIST01Part2()
b.get_datasets()
b.convert_datasets_to_tensors()
b.train_imgs.shape